# Vision Edge-AI와 3D Depth Camera

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 사용 이유
1. **정확한 깊이 인식**: 자율 주행, 로봇 공학, 물체 감지 등에서 정확한 깊이 정보가 필수.
2. **환경 인식**: 3D로 공간을 분석함으로써 보다 풍부한 정보를 얻기 용이,
3. **응용 분야**: 가상 현실(VR), 증강 현실(AR), 산업용 로봇 등 다양한 분야에서 사용.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

### OAK-D PRO의 자체 기능과 기술
- **IMU 센서**: 내장된 IMU 센서로 자세와 움직임을 추적할 수 있다.
- **AI 프로세싱**: 내장된 AI 칩으로 머신러닝 모델을 로컬에서 실행 가능.
- **다중 카메라**: 컬러 카메라와 두 개의 모노 카메라를 사용하여 스테레오 비전 지원.
- **ROS 지원**: 로봇 운영 체제(ROS)와의 통합 용이.

# 간단한 예제 실습

## Jetson Orin Nano에서 Camera module setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 단계
1. Jetson Orin Nano와 OAK-D PRO를 USB-C 케이블로 연결합니다.
2. Jetson Orin Nano의 터미널을 열고, `lsusb` 명령어로 카메라가 연결되었는지 확인합니다.
3. DepthAI Python 패키지를 설치합니다:
   ```bash
   pip install depthai
   ```
4. DepthAI 샘플 코드를 실행하여 연결 테스트:
   ```python
   import depthai as dai

   pipeline = dai.Pipeline()

   cam_rgb = pipeline.createColorCamera()
   cam_rgb.setPreviewSize(640, 480)
   cam_rgb.setInterleaved(False)
   cam_rgb.setFps(30)

   xout_preview = pipeline.createXLinkOut()
   xout_preview.setStreamName("rgb")

   cam_rgb.preview.link(xout_preview.input)

   with dai.Device(pipeline) as device:
       queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
       while True:
           frame = queue.get().getCvFrame()
           cv2.imshow("rgb", frame)
           if cv2.waitKey(1) == ord('q'):
               break
   ```

## Jetson Orin Nano에서 DeepStream + OAK-D PRO

### 설치 및 구성
1. DeepStream 설치:
   ```bash
   sudo apt-get install deepstream-6.1
   ```
2. OAK-D PRO와 DeepStream을 연결하기 위한 플러그인 설치:
   ```bash
   git clone https://github.com/NVIDIA-AI-IOT/deepstream_plugins
   cd deepstream_plugins
   make
   sudo make install
   ```

3. 기본 파이프라인 구성:
   - RTSP, USB 카메라, 파일 입력 모두 지원.

## TEST 3: 다중 파이프라인 구성 (RTSP, File, Camera, ...)

### 파이프라인 예제
1. **RTSP 스트림**:
   ```bash
   gst-launch-1.0 rtspsrc location=rtsp://<ip-address>:<port>/stream ! decodebin ! videoconvert ! autovideosink
   ```
2. **파일 스트림**:
   ```bash
   gst-launch-1.0 filesrc location=/path/to/video.mp4 ! decodebin ! videoconvert ! autovideosink
   ```
3. **USB 카메라 스트림**:
   ```bash
   gst-launch-1.0 v4l2src device=/dev/video0 ! decodebin ! videoconvert ! autovideosink
   ```

### TEST1-USBCAM : pipeline with USB camera input

```python
import gi
import cv2
import depthai as dai

gi.require_version('Gst', '1.0')
from gi.repository import Gst

# Initialize GStreamer
Gst.init(None)

pipeline = dai.Pipeline()

cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(640, 480)
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)

xout_preview = pipeline.createXLinkOut()
xout_preview.setStreamName("rgb")

cam_rgb.preview.link(xout_preview.input)

with dai.Device(pipeline) as device:
    queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    while True:
        frame = queue.get().getCvFrame()
        cv2.imshow("rgb", frame)
        if cv2.waitKey(1) == ord('q'):
            break
```
